<a href="https://colab.research.google.com/github/oxayavongsa/aai-590-capstone-mental-health/blob/main/User%20Interface%20Files/survey%20interface/mental_health_risk_predictor_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mental Health Risk Predictor (Demo Only)

In [1]:
#!pip install gradio openai joblib

In [14]:
import openai
import os
import gradio as gr
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load trained model pipeline
model = joblib.load("logistic_pipeline_model.pkl")

# Set your OpenAI key
#client = openai.OpenAI(api_key="ADD YOUR OPEN AI API KEY HERE")

In [3]:
# Extract PolynomialFeatures from the pipeline
poly = model.named_steps['poly']
base_feature_names = poly.feature_names_in_.tolist()
expanded_feature_names = poly.get_feature_names_out().tolist()
base_feature_names

['Gender',
 'family_history',
 'treatment',
 'Growing_Stress',
 'Mental_Health_History',
 'Mood_Swings',
 'Coping_Struggles',
 'Work_Interest']

### Chart Plotting Function

In [4]:
# Plotting function
def plot_risk_contributions(input_df, model):
    try:
        clf = model.named_steps['clf']
        coef = clf.coef_[0]
        input_array = input_df.values.flatten()
        contributions = coef * input_array

        contrib_df = pd.DataFrame({
            "Feature": input_df.columns,
            "Contribution": contributions
        })
        contrib_df["AbsContribution"] = np.abs(contrib_df["Contribution"])
        contrib_df = contrib_df.sort_values("AbsContribution", ascending=False).head(5)

        fig, ax = plt.subplots(figsize=(7, 4))
        bars = ax.bar(contrib_df["Feature"], contrib_df["AbsContribution"], color='skyblue')
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2.0, height + 0.01, f"{height:.2f}", ha='center', fontsize=9)

        ax.set_title("Top Risk Factor Contributions")
        ax.set_ylabel("Contribution Score")
        ax.set_xticks(range(len(contrib_df)))
        ax.set_xticklabels(contrib_df["Feature"], rotation=45)
        fig.tight_layout()
        plt.close(fig)

        return fig
    except Exception as e:
        print(f"Chart Error: {e}")
        return None

### Main Prediction Logic

In [12]:
# Main prediction logic
def predict_risk(diagnosis, interest_level, stress_level, swings, coping, treatment, family, gender):
    try:
        interest_map = {"Very low": 1, "Somewhat low": 3, "Neutral": 5, "Somewhat high": 7, "Very high": 9}
        stress_map = {"No increase": 1, "Slight increase": 3, "Moderate increase": 5, "Significant increase": 7, "Extreme increase": 9}

        feature_values = [
            {"Female": 0, "Male": 1, "Other": 2}[gender],
            1 if family == "Yes" else 0,
            1 if treatment == "Yes" else 0,
            stress_map[stress_level],
            1 if diagnosis == "Yes" else 0,
            1 if swings == "Yes" else 0,
            1 if coping == "Yes" else 0,
            interest_map[interest_level]
        ]

        column_names = [
            'Gender', 'family_history', 'treatment', 'Growing_Stress',
            'Mental_Health_History', 'Mood_Swings', 'Coping_Struggles', 'Work_Interest'
        ]
        input_df = pd.DataFrame([feature_values], columns=column_names)

        prediction = model.predict(input_df)[0]
        label_map = {
            0: ("Low Risk 😌", "#d4edda"),
            1: ("Moderate Risk 😐", "#fff3cd"),
            2: ("High Risk 😟", "#f8d7da")
        }
        plain_label = {0: "Low Risk", 1: "Moderate Risk", 2: "High Risk"}[prediction]

        recommendation_map = {
            "Low Risk": "You appear emotionally stable. Keep up healthy habits and self-care.",
            "Moderate Risk": "Try to maintain balance and consider periodic self-assessments or speaking with a counselor.",
            "High Risk": "We encourage reaching out to a licensed mental health professional for support."
        }

        risk_label, bg_color = label_map[prediction]
        recommendation = recommendation_map[plain_label]

        stress_score = stress_map[stress_level] / 9 * 100
        emotion_score = 85 if swings == "Yes" else 60
        social_score = 100 - interest_map[interest_level] * 10
        emotional_type = "Resilient but Overwhelmed" if prediction == 2 else (
            "Stable with Fluctuations" if prediction == 1 else "Emotionally Balanced")

        snapshot_html = f"""
        <div style='background-color:{bg_color}; border-radius:10px; padding:15px;'>
        <h4><b>🧠 Emotional Type:</b> {emotional_type}</h4>
        <ul>
        <li><b>Stress-Prone:</b> {stress_score:.0f}%</li>
        <li><b>Emotionally Aware:</b> {emotion_score}%</li>
        <li><b>Social Sensitivity:</b> {social_score}%</li>
        </ul>
        <i>You may feel deeply and prioritize others. Take space to breathe and be kind to yourself.</i>
        </div>
        """

        chart = plot_risk_contributions(input_df, model)
        return risk_label, recommendation, snapshot_html, chart

    except Exception as e:
        print(f"Exception: {str(e)}")
        return "Error", "Problem with your input.", f"Error: {str(e)}", None

### Build Gradio Interface

In [13]:
# Gradio UI
def build_interface():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("## 🧠 Mental Health Risk Predictor")
        gr.Markdown("Answer a few questions to get your mental health risk level, recommendation, and personality snapshot.")

        with gr.Row():
            diagnosis = gr.Radio(["Yes", "No"], label="Diagnosed with mental health condition?")
            interest = gr.Radio(["Very low", "Somewhat low", "Neutral", "Somewhat high", "Very high"], label="Interest in work or daily activities")
            stress = gr.Radio(["No increase", "Slight increase", "Moderate increase", "Significant increase", "Extreme increase"], label="Recent stress increase")

        with gr.Row():
            swings = gr.Radio(["Yes", "No"], label="Frequent mood swings?")
            coping = gr.Radio(["Yes", "No"], label="Struggling to cope with challenges?")
            treatment = gr.Radio(["Yes", "No"], label="Currently receiving treatment?")

        with gr.Row():
            family = gr.Radio(["Yes", "No"], label="Family history of mental health issues?")
            gender = gr.Radio(["Female", "Male", "Other"], label="Gender")

        submit = gr.Button("Assess My Risk", variant="primary")

        risk_box = gr.Textbox(label="Risk Level")
        reco_box = gr.Textbox(label="Recommendation")
        snapshot_box = gr.HTML(label="Personality Snapshot")
        chart_output = gr.Plot(label="Top Risk Contributors")

        submit.click(
            fn=predict_risk,
            inputs=[diagnosis, interest, stress, swings, coping, treatment, family, gender],
            outputs=[risk_box, reco_box, snapshot_box, chart_output]
        )
    return demo

# Launch app
demo = build_interface()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ea42613777003aa15b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
